In [1]:
from src.api import API
from src.dataloader import DataLoader
from src.prompter import prompter_factory

In [2]:
TOGETHER_API_KEY = "92a6ac4a8feb39c91b4a3f77219e9c452d927f5f4d543d5969cc11c210795719"
BASE_URL = "https://api.together.xyz"
ALLOWED_SYMPTOMS = ['anxiety', 'concentration problems', 'constipation', 'cough', 'diarrhea', 'fatigue', 'fever', 'headache', 'nausea', 'numbness and tingling', 'pain', 'poor appetite', 'rash', 'shortness of breath', 'trouble drinking fluids', 'vomiting', 'other']

Using the API class provides a modular approach to client definition.

In [3]:
api = API(api_key=TOGETHER_API_KEY, base_url=BASE_URL)
client = api.get_openai()

The DataLoader class loads the csv files. It verifies their existence and provides testing tools.

DataLoader allows for dataframe creation, with uniformization of the labels and column formatting.

In [4]:
dataloader = DataLoader(path="data/")
print(dataloader.list_csv_files())

['data/batch_1_gs.csv', 'data/batch_2_gs.csv', 'data/batch_3_gs.csv', 'data/batch_4_gs.csv', 'data/batch_5_gs.csv', 'data/batch_6_gs.csv', 'data/batch_7_gs.csv', 'data/batch_8_gs.csv', 'data/batch_9_gs.csv', 'data/batch_10_gs.csv']


In [5]:
df = dataloader.get_standardized_dataframe(context_col="Text Data",
                                           target_col="symptom_status_gs",
                                           keep_other_cols=True)

In [6]:
dataloader.check_symptoms_validity(allowed_symptoms=ALLOWED_SYMPTOMS,
                                   symptoms_col="symptom_detail_gs")

Symptoms in dataframe are valid.


In [7]:
df.describe()

,Context,Symptom Status_bd,Detailed Choices_bd,Source_x,Symptom Status_CVD,Detailed Choices_CVD,Source_y,Differences,Adjudication Decision,Pres_Abs_Diffs,Target,symptom_detail_gs,Symptom Status_KS,Detailed Choices_KS
count,550,268,189,550,550,382,550,128,128,25,550,380,282,185
unique,550,2,95,50,2,144,50,31,69,2,2,147,2,77
top,"D: How may I help you?\n\nP: Hi, umm, so I've ...",Positive,Pain,RES0022.txt,Positive,Pain,RES0022.txt,Other,pain,Negative;Positive,Positive,pain,Positive,Pain
freq,1,188,17,17,382,38,17,37,20,17,380,50,185,27


In [8]:
prompter = prompter_factory(prompter_type="binary",
                            client=client,
                            model="mistralai/Mixtral-8x7B-Instruct-v0.1")

In [9]:
prompt = "Are any medical symptoms mentioned in the transcript"
context = "i have a headache every time i see a cat and i hear voices that are not there"
print(prompter.generate_single(prompt=prompt, context=context))

status=True


In [10]:
prompter.generate(df=df,
                  prompt=prompt)

100%|██████████| 550/550 [00:49<00:00, 11.08it/s]


,Context,Symptom Status_bd,Detailed Choices_bd,Source_x,Symptom Status_CVD,Detailed Choices_CVD,Source_y,Differences,Adjudication Decision,Pres_Abs_Diffs,Target,symptom_detail_gs,Symptom Status_KS,Detailed Choices_KS,output_mistralai/Mixtral_8x7B_Instruct_v0_1
0,"D: How may I help you?\n\nP: Hi, umm, so I've ...",Positive,Fever;Other,RES0181.txt,Positive,Fever;Pain;Other,RES0181.txt,Pain,fever;other;trouble drinking fluids,NaN,Positive,fever;other;trouble drinking fluids,NaN,NaN,status=True
1,D: OK. And how about your appetite? Has that c...,Positive,Fever;Poor appetite;Other,RES0181.txt,Positive,Fever;Pain;Poor appetite;Other,RES0181.txt,Pain,fever;other;poor appetite,NaN,Positive,fever;other;poor appetite,NaN,NaN,status=True
2,D: OK. Have you had any any headaches?\n\nP: U...,Positive,Cough;Headache;Nausea;Pain;Shortness of breath...,RES0181.txt,Positive,Cough;Headache;Nausea;Pain;Shortness of breath...,RES0181.txt,NaN,NaN,NaN,Positive,cough;headache;nausea;other;pain;shortness of ...,NaN,NaN,status=False
3,D: Have you had any abdominal pain?\n\nP: No. ...,Positive,Constipation;Diarrhea;Fatigue;Pain;Rash;Other,RES0181.txt,Positive,Constipation;Diarrhea;Fatigue;Pain;Rash;Other,RES0181.txt,NaN,NaN,NaN,Positive,constipation;diarrhea;fatigue;other;pain;rash,NaN,NaN,status=False
4,"P: I use an insulin pump, yeah.\n\nD: OK, exce...",Positive,Anxiety;Other,RES0181.txt,Positive,Anxiety,RES0181.txt,Other,anxiety,NaN,Positive,anxiety,NaN,NaN,status=False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,"D: OK, and do you feel, is there any pain to p...",NaN,NaN,MSK0009.txt,Positive,Pain,MSK0009.txt,NaN,NaN,NaN,Positive,pain,Positive,Pain,status=True
546,"P: No, I think that's all.",NaN,NaN,MSK0009.txt,Negative,NaN,MSK0009.txt,NaN,NaN,NaN,Negative,NaN,Negative,NaN,status=False
547,D: OK yeah alright so with the. I fall 2 days ...,NaN,NaN,MSK0009.txt,Positive,Fever;Pain;Other,MSK0009.txt,Fever,fever;other;pain,NaN,Positive,fever;other;pain,Positive,Pain;Other,status=True
548,Ray of the right knee to to start OK.,NaN,NaN,MSK0009.txt,Negative,NaN,MSK0009.txt,NaN,NaN,NaN,Negative,NaN,Negative,NaN,status=True
